# Word Embeding

# gensim



Train Model

In [ ]:
from gensim.models import Word2Vec

# Train Model
def train_model(inputFilePath, modelPath):
    # size : each vector lenght
    # window : num of each window size
    # min_count : number of min word repeat to consider in training
    # workers : num of multiprocess task
    model = Word2Vec(LineSentence(inputFilePath), size=500, window=7, min_count=15, workers=8)
    # Training model finished
    # saving model
    model.save(modelPath)

# Clear Corpus
def clear_data(inputFilePath, outPutFilePath):
    # Open files
    with open(inputFilePath, "r", encoding="utf-8", errors="replace") as ifp, open(outPutFilePath, 'a',encoding="utf-8") as ofp:
        # Iterate over Corpus
        for line in ifp:
            # Ignore non utf-8 char
            line = bytes(line, 'utf-8').decode('utf-8', 'ignore')
            ofp.write(line)
    print("Corpus Cleaned")


# Main program
if __name__ == "__main__":
    # Set path
    inputFilePath = "Miras.text"
    cleanFilePath = "cleanMiras.txt"
    modelPath = "model"

    # Clean and Train
    clear_data(inputFilePath, cleanFilePath)
    train_model(cleanFilePath, modelPath)

Run time : 8 cpu 32g RAM, 100 Min

Load Model

In [6]:
import gensim
# Load and see Model
def load_model():
    model = gensim.models.Word2Vec.load('model')
    print(model.wv.most_similar(['درخت']))
    model1 = gensim.models.Word2Vec.load('model200')
    print(model1.wv.most_similar(['درخت']))

if __name__ == "__main__":
    # Run function
    load_model()

C:\ProgramData\Anaconda3\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
C:\ProgramData\Anaconda3\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('درختان', 0.7574159502983093), ('درختی', 0.6967282295227051), ('درختانی', 0.6075932383537292), ('درختهای', 0.6055514812469482), ('درختي', 0.6005440354347229), ('ممرز', 0.5607376098632812), ('کُنار', 0.5591282248497009), ('درختچه', 0.5504060387611389), ('درختها', 0.5439611673355103), ('سرخدار', 0.5391978621482849)]
[('درختان', 0.7944933772087097), ('درختی', 0.7790977954864502), ('سرخدار', 0.7013828754425049), ('بارانک', 0.6784353852272034), ('تنگرس', 0.6735241413116455), ('درختهای', 0.6730005741119385), ('کُنار', 0.6659190058708191), ('ممرز', 0.665337085723877), ('صنوبر', 0.6644573211669922), ('سدر', 0.6556769609451294)]


Using PCA

In [15]:
import os
from sklearn.decomposition import PCA
from matplotlib import pyplot

def viewModel():
    # Load model
    model = gensim.models.Word2Vec.load(os.path.abspath(r"C:\Users\saeed\Desktop\PRESENT\model"))
    X = model[model.wv.vocab]
    # Define PCA
    pca = PCA(n_components=2)
    # Fit PCA
    result = pca.fit_transform(X)

    # create a scatter plot of the projection
    pyplot.scatter(result[:, 0], result[:, 1])
    # List words
    words = list(model.wv.vocab)
    for i, word in enumerate(words):
        pyplot.annotate(word, xy=(result[i, 0], result[i, 1]))
    # Save fig
    pyplot.savefig('model.jpg')
    pyplot.show()

if __name__ == "__main__":
    # Run Function
    viewModel()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


<Figure size 640x480 with 1 Axes>

Tensor Representation

In [17]:
# encoding: utf-8

import sys, os
from gensim.models import Word2Vec
import tensorflow as tf
import numpy as np
from tensorflow.contrib.tensorboard.plugins import projector


def visualize(model, output_path):
    meta_file = "w2x_metadata.tsv"
    placeholder = np.zeros((len(model.wv.index2word), 200))

    with open(os.path.join(output_path,meta_file), 'wb') as file_metadata:
        for i, word in enumerate(model.wv.index2word):
            placeholder[i] = model[word]
            # temporary solution for https://github.com/tensorflow/tensorflow/issues/9094
            if word == '':
                print("Emply Line, should replecaed by any thing else, or will cause a bug of tensorboard")
                file_metadata.write("{0}".format('<Empty Line>').encode('utf-8') + b'\n')
            else:
                file_metadata.write("{0}".format(word).encode('utf-8') + b'\n')

    # define the model without training
    sess = tf.InteractiveSession()

    embedding = tf.Variable(placeholder, trainable=False, name='w2x_metadata')
    tf.global_variables_initializer().run()

    saver = tf.train.Saver()
    writer = tf.summary.FileWriter(output_path, sess.graph)

    # adding into projector
    config = projector.ProjectorConfig()
    embed = config.embeddings.add()
    embed.tensor_name = 'w2x_metadata'
    embed.metadata_path = meta_file

    # Specify the width and height of a single thumbnail.
    projector.visualize_embeddings(writer, config)
    saver.save(sess, os.path.join(output_path,'w2x_metadata.ckpt'))
    print('Run `tensorboard --logdir={0}` to run visualize result on tensorboard'.format(output_path))

if __name__ == "__main__":

    model_path = os.path.abspath(r"C:\Users\saeed\Desktop\PRESENT\model200")
    output_path = os.path.abspath(r"C:\Users\saeed\Desktop\PRESENT")
    model = Word2Vec.load(model_path)
    visualize(model, output_path)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  app.launch_new_instance()


Run `tensorboard --logdir=C:\Users\saeed\Desktop\PRESENT` to run visualize result on tensorboard
